In [2]:
import os
os.environ["OPENAI_API_KEY"] = key  # reemplaza con tu clave real si es necesario




## Arquitecturas Cognitivas con LangGraph

Hasta ahora, hemos visto las características más comunes de las aplicaciones con modelos de lenguaje (LLM):

- Técnicas de creación de prompts en el Prefacio y el Capítulo 1  
- RAG en los Capítulos 2 y 3  
- Memoria en el Capítulo 4  

La siguiente pregunta debería ser: ¿cómo ensamblamos estas piezas en una aplicación coherente que logre el objetivo que nos propusimos? Para hacer un paralelo con el mundo de los ladrillos y el cemento, una piscina y una casa de un solo piso están construidas con los mismos materiales, pero obviamente cumplen propósitos muy diferentes. Lo que las hace adecuadas para sus distintos fines es el plan sobre cómo se combinan esos materiales —es decir, su arquitectura. Lo mismo ocurre al construir aplicaciones con LLM. Las decisiones más importantes que debes tomar tienen que ver con cómo ensamblar los diferentes componentes a tu disposición (como RAG, técnicas de prompting, memoria) en algo que cumpla tu propósito.

Antes de ver arquitecturas específicas, veamos un ejemplo. Cualquier aplicación con LLM que construyas comenzará con un propósito: lo que la aplicación está diseñada para hacer. Supongamos que quieres construir un asistente de correo electrónico —una aplicación con LLM que lea tus correos antes que tú y tenga como objetivo reducir la cantidad de correos que necesitas revisar. La aplicación podría hacer esto archivando algunos correos irrelevantes, respondiendo directamente a otros, y marcando algunos como merecedores de tu atención más adelante.

Probablemente también querrías que la aplicación esté limitada por ciertas restricciones en sus acciones. Enumerar esas restricciones ayuda muchísimo, ya que te ayudarán a definir la arquitectura adecuada. El Capítulo 8 aborda estas restricciones con más detalle y cómo trabajar con ellas. Para este asistente de correo electrónico hipotético, digamos que nos gustaría que hiciera lo siguiente:

- Minimizar la cantidad de veces que te interrumpe (después de todo, la idea es ahorrar tiempo).  
- Evitar que tus corresponsales reciban una respuesta que tú nunca habrías enviado.  

Esto apunta a un conflicto clave al construir aplicaciones con LLM: el equilibrio entre **agencia** (la capacidad de actuar de forma autónoma) y **confiabilidad** (el grado en que puedes confiar en sus respuestas). Intuitivamente, el asistente será más útil si actúa más sin tu intervención, pero si se le da demasiada libertad, inevitablemente enviará mensajes que preferirías no haber enviado.

Una forma de describir el grado de autonomía de una aplicación con LLM es evaluar cuánta parte del comportamiento de la aplicación está determinada por un LLM (en vez de por código):

- Hacer que un LLM decida el resultado de un paso (por ejemplo, redactar una respuesta a un correo).  
- Hacer que un LLM decida cuál es el siguiente paso a tomar (por ejemplo, ante un nuevo correo, decidir entre archivar, responder o marcar para revisión).  
- Hacer que un LLM decida qué pasos están disponibles para tomar (por ejemplo, que el LLM escriba código que ejecute una acción dinámica que no fue preprogramada en la aplicación).  

Podemos clasificar varias recetas populares para construir aplicaciones con LLM en función de dónde caen en este espectro de autonomía —es decir, qué tareas de las tres anteriores son manejadas por un LLM y cuáles siguen en manos del desarrollador o del usuario. Estas recetas se pueden llamar **arquitecturas cognitivas**. En el campo de la inteligencia artificial, el término arquitectura cognitiva se ha usado durante mucho tiempo para denotar modelos de razonamiento humano (y sus implementaciones en computadoras). Una **arquitectura cognitiva con LLM** (el término fue aplicado a los LLM por primera vez, hasta donde sabemos, en un artículo¹) se puede definir como una receta de los pasos que debe seguir una aplicación con LLM (ver Figura 5-1). Un paso es, por ejemplo, recuperar documentos relevantes (RAG), o llamar a un LLM con un prompt de cadena de pensamiento (*chain-of-thought*).

**Figura 5-1. Arquitecturas cognitivas para aplicaciones con LLM**

Ahora veamos cada una de las principales arquitecturas, o recetas, que puedes usar al construir tu aplicación (como se muestra en la Figura 5-1):

### 0: Código
Esto no es una arquitectura cognitiva con LLM (por eso la numeramos como 0), ya que no utiliza LLMs en absoluto. Puedes pensar en esto como el software tradicional que estás acostumbrado a escribir. La primera arquitectura interesante (para este libro, al menos) es la siguiente.

### 1: Llamada a LLM
Esta es la mayoría de los ejemplos que hemos visto en el libro hasta ahora, con una sola llamada a un LLM. Es útil principalmente cuando forma parte de una aplicación más grande que usa un LLM para lograr una tarea específica, como traducir o resumir un texto.

### 2: Cadena
El siguiente nivel consiste en el uso de múltiples llamadas a LLM en una secuencia predefinida. Por ejemplo, una aplicación de texto a SQL (que recibe como entrada una descripción en lenguaje natural de algún cálculo a realizar sobre una base de datos) podría usar dos llamadas a LLM en secuencia:

- Una llamada a LLM para generar una consulta SQL, a partir de la consulta en lenguaje natural proporcionada por el usuario y una descripción del contenido de la base de datos proporcionada por el desarrollador.  
- Otra llamada a LLM para escribir una explicación de la consulta apropiada para un usuario no técnico, dada la consulta generada en la llamada anterior. Esta se puede usar para que el usuario verifique si la consulta generada coincide con su solicitud.

### 3: Enrutador
Este siguiente paso consiste en usar el LLM para definir la secuencia de pasos a seguir. Es decir, mientras que la arquitectura en cadena siempre ejecuta una secuencia estática de pasos (por muchos que sean) determinada por el desarrollador, la arquitectura de enrutador se caracteriza por usar un LLM para elegir entre ciertos pasos predefinidos. Un ejemplo sería una aplicación RAG con múltiples índices de documentos de diferentes dominios, con los siguientes pasos:

- Una llamada a LLM para elegir cuál de los índices disponibles usar, dado la consulta proporcionada por el usuario y la descripción de los índices proporcionada por el desarrollador.  
- Un paso de recuperación que consulta el índice elegido para obtener los documentos más relevantes para la consulta del usuario.  
- Otra llamada a LLM para generar una respuesta, dada la consulta del usuario y la lista de documentos relevantes recuperados del índice.  

Hasta aquí llega este capítulo. Hablaremos de cada una de estas arquitecturas más adelante. Los próximos capítulos discuten las **arquitecturas agenticas**, que hacen un uso aún mayor de los LLMs. Pero primero, hablemos de mejores herramientas para ayudarnos en este camino.





### Arquitectura #1: Llamada a LLM

Como ejemplo de la arquitectura basada en una llamada a un LLM, volveremos al chatbot que creamos en el Capítulo 4. Este chatbot responderá directamente a los mensajes del usuario.

Comienza creando un `StateGraph`, al cual le agregaremos un nodo para representar la llamada al LLM:

También podemos dibujar una representación visual del grafo:

El grafo que acabamos de crear se ve como en la Figura 5-2.

Puedes ejecutarlo con el método `stream()` que ya viste en capítulos anteriores:

La salida:

```json
{ "chatbot": { "messages": [AIMessage("¿En qué puedo ayudarte?")] } }
```

Observa cómo la entrada al grafo tiene el mismo formato que el objeto `State` que definimos anteriormente; es decir, enviamos una lista de mensajes dentro de la clave `messages` de un diccionario.

Esta es la arquitectura más simple posible para utilizar un modelo de lenguaje (LLM), lo cual no significa que no deba usarse. Aquí algunos ejemplos de dónde podrías verla en acción en productos populares:

- Funciones impulsadas por IA como **resumir** o **traducir** (como las que puedes encontrar en Notion, un software de escritura popular) pueden estar impulsadas por una sola llamada a un LLM.
- La generación simple de consultas SQL también puede ser alimentada por una sola llamada a un LLM, dependiendo de la experiencia de usuario y el tipo de usuario objetivo que tenga en mente el desarrollador.

---

### Solo código en Python:

```python
from typing import Annotated, TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage


model = ChatOpenAI()

class State(TypedDict):
    # Los mensajes tienen el tipo "lista". La función `add_messages` 
    # en la anotación define cómo debe actualizarse este estado 
    # (en este caso, añade nuevos mensajes a la lista en lugar de reemplazarlos)
    messages: Annotated[list, add_messages]

def chatbot(state: State):
    answer = model.invoke(state["messages"])
    return {"messages": [answer]}

builder = StateGraph(State)
builder.add_node("chatbot", chatbot)
builder.add_edge(START, 'chatbot')
builder.add_edge('chatbot', END)

graph = builder.compile()

# Dibujar representación del grafo
graph.get_graph().draw_mermaid_png()

# Ejecutar usando el método stream
input = {"messages": [HumanMessage('hi!')]}
for chunk in graph.stream(input):
    print(chunk)




### Arquitectura #2: Cadena (Chain)

Esta siguiente arquitectura amplía la anterior usando múltiples llamadas a LLM, en una secuencia predefinida (es decir, distintas ejecuciones de la aplicación siguen la misma secuencia de llamadas LLM, aunque con diferentes entradas y resultados).

Tomemos como ejemplo una aplicación de texto a SQL, que recibe como entrada del usuario una descripción en lenguaje natural de algún cálculo que quiere realizar sobre una base de datos. Mencionamos antes que esto podría lograrse con una sola llamada a un LLM, para generar una consulta SQL, pero podemos crear una aplicación más sofisticada usando múltiples llamadas LLM en secuencia. Algunos autores llaman a esta arquitectura *ingeniería de flujos*.

Primero describamos el flujo con palabras:

- Una llamada a LLM para generar una consulta SQL a partir de la pregunta en lenguaje natural proporcionada por el usuario, y una descripción del contenido de la base de datos proporcionada por el desarrollador.
- Otra llamada a LLM para escribir una explicación de la consulta, adecuada para un usuario no técnico, con base en la consulta generada en la llamada anterior. Esto puede usarse para permitir que el usuario verifique si la consulta generada coincide con su solicitud.

También podrías extender esto aún más (aunque no lo haremos aquí) con pasos adicionales después de los dos anteriores:

- Ejecutar la consulta contra la base de datos, lo cual devuelve una tabla bidimensional.
- Usar una tercera llamada a LLM para resumir los resultados en una respuesta textual a la pregunta original del usuario.

Y ahora, implementemos esto con **LangGraph**:

---

### Código en Python:



In [3]:
from typing import Annotated, TypedDict

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages

# Modelo con baja temperatura para generar consultas SQL
model_low_temp = ChatOpenAI(temperature=0.1)
# Modelo con mayor temperatura para generar explicaciones en lenguaje natural
model_high_temp = ChatOpenAI(temperature=0.7)

class State(TypedDict):
    # Historial de conversación
    messages: Annotated[list, add_messages]
    # Entrada del usuario
    user_query: str
    # Salidas
    sql_query: str
    sql_explanation: str

class Input(TypedDict):
    user_query: str

class Output(TypedDict):
    sql_query: str
    sql_explanation: str

generate_prompt = SystemMessage(
    """Eres un analista de datos útil que genera consultas SQL para los usuarios 
    en función de sus preguntas."""
)

def generate_sql(state: State) -> State:
    user_message = HumanMessage(state["user_query"])
    messages = [generate_prompt, *state["messages"], user_message]
    res = model_low_temp.invoke(messages)
    return {
        "sql_query": res.content,
        "messages": [user_message, res],
    }

explain_prompt = SystemMessage(
    "Eres un analista de datos útil que explica consultas SQL a los usuarios."
)

def explain_sql(state: State) -> State:
    messages = [
        explain_prompt,
        *state["messages"],
    ]
    res = model_high_temp.invoke(messages)
    return {
        "sql_explanation": res.content,
        "messages": res,
    }

builder = StateGraph(State, input=Input, output=Output)
builder.add_node("generate_sql", generate_sql)
builder.add_node("explain_sql", explain_sql)
builder.add_edge(START, "generate_sql")
builder.add_edge("generate_sql", "explain_sql")
builder.add_edge("explain_sql", END)

graph = builder.compile()


In [4]:
graph.invoke({
  "user_query": "What is the total sales for each product?"
})

{'sql_query': 'Para obtener el total de ventas de cada producto, puedes utilizar la siguiente consulta SQL:\n\n```sql\nSELECT product_name, SUM(sales_amount) AS total_sales\nFROM sales\nGROUP BY product_name;\n```\n\nEsta consulta te mostrará el nombre de cada producto junto con el total de ventas de ese producto. Asegúrate de reemplazar "product_name" y "sales_amount" con los nombres reales de las columnas en tu base de datos.',
 'sql_explanation': '¿Te gustaría saber algo más sobre consultas SQL?'}



### Arquitectura #3: Enrutador (Router)

Esta siguiente arquitectura sube en la escala de autonomía al asignar a los LLMs la siguiente responsabilidad que mencionamos antes: decidir el siguiente paso a tomar. Es decir, mientras que la arquitectura de cadena siempre ejecuta una secuencia estática de pasos (cuantos más pasos, mejor), la arquitectura de enrutador se caracteriza por utilizar un LLM para elegir entre ciertos pasos predefinidos.

Tomemos como ejemplo una aplicación **RAG** con acceso a múltiples índices de documentos de diferentes dominios (consulta el Capítulo 2 para más detalles sobre indexación). Usualmente, puedes obtener un mejor rendimiento de los LLMs evitando la inclusión de información irrelevante en el prompt. Por lo tanto, al construir esta aplicación, debemos intentar seleccionar el índice adecuado para cada consulta y usar solo ese.

El desarrollo clave en esta arquitectura es utilizar un LLM para tomar esta decisión, utilizando efectivamente un LLM para evaluar cada consulta entrante y decidir qué índice usar para esa consulta en particular.

**NOTA**  
Antes de la llegada de los LLMs, la forma habitual de resolver este problema sería construir un modelo de clasificación utilizando técnicas de ML y un conjunto de datos que mapea las consultas de los usuarios al índice correcto. Esto podría resultar bastante desafiante, ya que requiere lo siguiente:

- Armar ese conjunto de datos manualmente.
- Generar suficientes características (atributos cuantitativos) de cada consulta de usuario para habilitar el entrenamiento de un clasificador para la tarea.

Los LLMs, dado su codificado del lenguaje humano, pueden servir efectivamente como este clasificador con cero o muy pocos ejemplos o entrenamiento adicional.

Primero, describamos el flujo en palabras:

1. Una llamada a LLM para elegir qué de los índices disponibles usar, dada la consulta proporcionada por el usuario, y la descripción del índice proporcionada por el desarrollador.
2. Un paso de recuperación que consulta el índice elegido para los documentos más relevantes para la consulta del usuario.
3. Otra llamada a LLM para generar una respuesta, dada la consulta del usuario y la lista de documentos relevantes obtenidos del índice.

Y ahora, implementémoslo con **LangGraph**:

---

### Solo código en Python:


In [7]:
from typing import Annotated, Literal, TypedDict

from langchain_core.documents import Document
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.vectorstores.in_memory import InMemoryVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages

embeddings = OpenAIEmbeddings()
# útil para generar consultas SQL
model_low_temp = ChatOpenAI(temperature=0.1)
# útil para generar salidas en lenguaje natural
model_high_temp = ChatOpenAI(temperature=0.7)

class State(TypedDict):
    # para rastrear el historial de conversación
    messages: Annotated[list, add_messages]
    # entrada
    user_query: str
    # salida
    domain: Literal["records", "insurance"]
    documents: list[Document]
    answer: str

class Input(TypedDict):
    user_query: str

class Output(TypedDict):
    documents: list[Document]
    answer: str

# referencia al Capítulo 2 sobre cómo llenar un vector store con documentos
medical_records_store = InMemoryVectorStore.from_documents([], embeddings)
medical_records_retriever = medical_records_store.as_retriever()

insurance_faqs_store = InMemoryVectorStore.from_documents([], embeddings)
insurance_faqs_retriever = insurance_faqs_store.as_retriever()

router_prompt = SystemMessage(
    """Debes decidir a qué dominio enrutar la consulta del usuario. Tienes dos 
        dominios para elegir:
          - records: contiene los registros médicos del paciente, como 
          diagnóstico, tratamiento y prescripciones.
          - insurance: contiene preguntas frecuentes sobre políticas de 
          seguros, reclamaciones y cobertura.

Solo devuelve el nombre del dominio."""
)

def router_node(state: State) -> State:
    user_message = HumanMessage(state["user_query"])
    messages = [router_prompt, *state["messages"], user_message]
    res = model_low_temp.invoke(messages)
    return {
        "domain": res.content,
        "messages": [user_message, res],
    }

def pick_retriever(
    state: State,
) -> Literal["retrieve_medical_records", "retrieve_insurance_faqs"]:
    if state["domain"] == "records":
        return "retrieve_medical_records"
    else:
        return "retrieve_insurance_faqs"

def retrieve_medical_records(state: State) -> State:
    documents = medical_records_retriever.invoke(state["user_query"])
    return {
        "documents": documents,
    }

def retrieve_insurance_faqs(state: State) -> State:
    documents = insurance_faqs_retriever.invoke(state["user_query"])
    return {
        "documents": documents,
    }

medical_records_prompt = SystemMessage(
    """Eres un chatbot médico útil que responde preguntas basadas en los 
        registros médicos del paciente, como diagnóstico, tratamiento y 
        prescripciones."""
)

insurance_faqs_prompt = SystemMessage(
    """Eres un chatbot de seguros útil que responde preguntas frecuentes sobre 
        pólizas de seguros, reclamaciones y cobertura."""
)

def generate_answer(state: State) -> State:
    if state["domain"] == "records":
        prompt = medical_records_prompt
    else:
        prompt = insurance_faqs_prompt
    messages = [
        prompt,
        *state["messages"],
        HumanMessage(f"Documentos: {state['documents']}"),
    ]
    res = model_high_temp.invoke(messages)
    return {
        "answer": res.content,
        "messages": res,
    }

builder = StateGraph(State, input=Input, output=Output)
builder.add_node("router", router_node)
builder.add_node("retrieve_medical_records", retrieve_medical_records)
builder.add_node("retrieve_insurance_faqs", retrieve_insurance_faqs)
builder.add_node("generate_answer", generate_answer)
builder.add_edge(START, "router")
builder.add_conditional_edges("router", pick_retriever)
builder.add_edge("retrieve_medical_records", "generate_answer")
builder.add_edge("retrieve_insurance_faqs", "generate_answer")
builder.add_edge("generate_answer", END)

graph = builder.compile()


In [8]:
input = {
    "user_query": "Am I covered for COVID-19 treatment?"
}
for c in graph.stream(input):
    print(c)

{'router': {'domain': 'insurance', 'messages': [HumanMessage(content='Am I covered for COVID-19 treatment?', additional_kwargs={}, response_metadata={}, id='36e25d15-1a8b-4381-91aa-b248d8f4402a'), AIMessage(content='insurance', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 104, 'total_tokens': 106, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BNoMliA7ydCVGVC1nfnbodxsfbv7Z', 'finish_reason': 'stop', 'logprobs': None}, id='run-69c0b1dc-2ef1-4aee-9a08-831590befe2a-0', usage_metadata={'input_tokens': 104, 'output_tokens': 2, 'total_tokens': 106, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
{'retrieve_insurance_faqs': {'do